# Функторы и монады  
<br>
![](http://imgs.xkcd.com/comics/hypotheticals.png)
<br>

### Функторы и с чем их едят

## Что такое ковариантный функтор

>С точки зрения «синтаксического подхода», ковариантным функтором является всякий тип (назовем его X) имеющий type parameter (назовем его T) с методом, который имеет следующую сигнатуру (назовем его map)

```
trait X[T] {
  def map(f: T => R): X[R]
}
```
<br>
«Синтаксический подход» хорош тем, что он позволяет свести в общую схему многие категориальные конструкции
<br>
```
trait X[T] {
  // ковариант
  def map[R](f: T => R): X[R]

  // контрвариант
  def contramap[R](f: R => T): X[R]

  // инвариант
  def xmap[R](f: (T => R, R => T)): X[R]

  // апликативный функтор
  def apply[R](f: X[T => R]): X[R]

  // монада
  def flatMap[R](f: T => X[R]): X[R]

  // комонада
  def coflatMap[R](f: X[T] => R): X[R]
}
```


| функтор  | аргументы  |
|---|---|
|ковариант| A => B  |
|контрвариант|  B => A |
|exponential| (A => B, B => A)  |
|апликативный|  F[A => B] |
|монада| A => F[B]  |
|комонада| F[A] => B  |

Функтор - это контейнер. Он может хранить либо ничего, либо само значение
<br><br>
![](https://hsto.org/storage2/be0/182/2de/be01822de6f660845c952b2b4fa7edb6.png)
<br>

Примеры ковариантных функторов из стандартной библиотеки scala

## Option

In [1]:
// Наш первый пациент
Option

res0: Option.type = scala.Option$@61a71e63

Одна из первых монад, которую знают многие программисты - это Option. Так же известная как Maybe монада в Haskell и Optional в Java 8. Вы можете думать об этой монаде, как о коллекции, которая состоит из одного элемента. Но при этом она может либо содержать его, либо не содержать. Таким образом вы можете оборачивать результат в эту монаду. Даже тот результат, который может зафейлиться. Если результат не зафейлится, он вернёт значение и это значение вернётся обёрнутой в монаду. В противном случае там будет лежать None. Мы можем изменять монаду Option с помощью функции map.

In [2]:
val three = Option(3)

three: Option[Int] = Some(3)

In [3]:
val twelve = three map (_ * 4)

twelve: Option[Int] = Some(12)

Но когда мы захотим положить несколько значений в эту монаду, у нас получится Option в Option

```
Option[Option[Int]]
```

In [4]:
val four = Option(4)

four: Option[Int] = Some(4)

In [5]:
val twelveB = three map (i => four map (i * _))

twelveB: Option[Option[Int]] = Some(Some(12))

^ тут мы получили Option в Option, так сказать, монада на монаде. Конечно, это не то, что мы хотим, поэтому давайте попробуем использовать flatMap

![](https://hsto.org/storage2/983/73b/546/98373b54695813ce070a7fc782ca8a35.png)

In [6]:
val twelveC = three flatMap (i => four map (i * _))

twelveC: Option[Int] = Some(12)

flatMap — парень с улицы, flatMap знает толк в контекстах. Уж он-то в курсе, как применить функцию к упакованному в контекст значению.

In [7]:
val twelveD = for {
  i <- three
  j <- four
} yield (i * j)

twelveD: Option[Int] = Some(12)

Не забудем о том, что монада заботится о том, что наше значение может быть и пустым

In [8]:
val oops: Option[Int] = None

oops: Option[Int] = None

![](https://hsto.org/storage2/788/3ad/937/7883ad93713ea4406c129d54f83659d8.png)

In [9]:
val oopsB = for {
  i <- three
  j <- oops
} yield (i * j)

oopsB: Option[Int] = None

In [10]:
val oopsC = for {
  i <- oops
  j <- four
} yield (i * j)

oopsC: Option[Int] = None

![](https://hsto.org/storage2/238/88c/8ed/23888c8ed4ae153e5c4d62321fec63c1.png)
<p style="font-size: 10px">Билл О'Рейли ничегошеньки не смыслит в функторе Maybe </p>

H1: Option — «как бы контейнер» на один элемент, где может что-то лежит (Some), а может и нет (None)

## List

In [11]:
// Наш второй пациент
List

res10: List.type = scala.collection.immutable.List$@5e63475

А вот ещё один пример: что происходит, когда вы применяете функцию к списку?

![](https://hsto.org/storage2/5ba/e29/3d1/5bae293d123881aa5920d3e7e9e1d039.png)

Списки тоже функторы!

H2: List — контейнер, в котором может быть 0...N элементов

## Future

In [12]:
import scala.concurrent.duration._
import scala.concurrent.{Future,ExecutionContext,Await}
import ExecutionContext.Implicits.global

import scala.concurrent.duration._

import scala.concurrent.{Future,ExecutionContext,Await}

import ExecutionContext.Implicits.global

In [13]:
// Третий на подходе
Future

res12: Future.type = scala.concurrent.Future$@ad4eb67

Существует монада для параллельного вычесления и асинхронности. Future монада существует для обёртки медленных вычислений, для того чтобы засунуть их в другой поток. Если вы зовёте Future монаду - она возращается тут же, позволяя вызывать поток, пока дополнительные потоки делают вычисления. Т.е. когда мы вызываем Future - она ещё не содержит значение, но через какое-то время оно (возможно) там появится.

Создадим 2 монады Future, каждая из которых будет спать 10 секунд. В главном потоке просуммируем f1 И f2. 

In [14]:
val f1=Future{
  Thread.sleep(10000)
  1 }

f1: Future[Int] = Future(<not completed>)

In [15]:
val f2=Future{
  Thread.sleep(10000)
  2 }

f2: Future[Int] = Future(<not completed>)

Заметим, что f1 и f2 выполняться вместе за 10 секунд в разных потоках.

In [16]:
val f3=for {
  v1 <- f1
  v2 <- f2
  } yield (v1+v2)

f3: Future[Int] = Future(<not completed>)

In [17]:
println(Await.result(f3, 30.seconds))

3


H3: Future — «как бы контейнер» на один элемент, где может что-то уже лежит, может будет лежать, или уже лежит исключение, или будет лежать исключение или никогда ничего лежать не будет.

## Try

In [20]:
import scala.util.Try

import scala.util.Try

In [22]:
// Ох, сколько больных развелось
Try

res21: Try.type = scala.util.Try$@120f1e89

Try монада существует для того, чтобы не оборачивать ошибки с помощью Either. Об Either поговорим чуть попозже, потому что это BiFunctor. Основная идея в том, что у вас лежит либо элемент, либо ошибка.

In [27]:
// Создадим функцию, которая будет возвращать монаду
def f(x: Int, y: Int): Try[Int] = Try(x / y)

defined function f

In [28]:
f(1, 1)

res27: Try[Int] = Success(1)

In [29]:
f(1, 0)

res28: Try[Int] = Failure(java.lang.ArithmeticException: / by zero)

Монада, как и было сказанно выше принимает 2 значения. Либо ```Success``` либо ```Failure```

In [32]:
// Узнаем, всё ли хорошо
f(1, 1).isSuccess

res31: Boolean = true

Ковариантный функтор — это не просто наличие метода с определенной сигнатурой, это также выполнение двух правил. Математики тут обычно отсылают к теории категорий, и говорят, что эти правила — следствие того, что функтор — это [гомоморфизм категорий](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%82%D0%BE%D1%80_(%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0)), то есть отображение категории в категорию, сохраняющее их структуру (а частью структуры категории являются единичный элемент-стрелка (правило Identity Law) и правила композиции стрелок (правило Composition law)). 

## Правила ковариантного функтора

### Identity Law

Для любого ковариантного функтора 'fun' должно тождественно выполняться следующее правило IdentityLaw.case0(fun) — то же самое что и IdentityLaw.case1(fun).

### Composition Law

Для любого ковариантного функтора 'fun[T]' и любых функций 'f: ' и 'g' должно тождественно выполняться следующее правило CompositionLaw.case0(fun) — то же самое что и CompositionLaw.case1(fun).

То есть произвольный функтор-контейнер, который последовательно отображают функцией 'f' и потом функцией 'g' эквивалентен тому, что мы строим новую функцию-композицию функций f и g (f andThen g) и отображаем один раз.


## Ковариантный функтор: используем для оптимизации

Давайте рассмотрим пример, демонстрирующий пользу от аксиом ковариантного функтора.

В качестве отображений рассмотрим линейные функции над целыми числами

In [35]:
case class LinFun(a: Int, b: Int) {
  def apply(k: Int): Int = a * k + b
  def andThen[A](that: LinFun): LinFun = LinFun(this.a * that.a, that.a * this.b + that.b)
}

defined class LinFun

Вместо самых общих функций вида T=>R я буду использовать их подмножество — линейные функции над Int, так как в отличии от общего вида я умею строить композиции линейных функций в явном виде.

В качестве функтора рассмотрю рекурсивный контейнер типа односвязный список целых чисел (Int)

In [39]:
LinFun(1, 3).apply(10)

res38: Int = 13

In [40]:
sealed trait IntSeq {
  def map(f: LinFun): IntSeq
}
case class Node(value: Int, tail: IntSeq) extends IntSeq {
  override def map(f: LinFun): IntSeq = Node(f(value), tail.map(f))
}
case object Last extends IntSeq {
  override def map(f: LinFun): IntSeq = Last
}

defined trait IntSeq
defined class Node
defined object Last

А теперь — демонстрация

In [44]:
val seq = Node(0, Node(1, Node(2, Node(3, Last))))
val f = LinFun(2, 3) // k => 2 * k + 3
val g = LinFun(4, 5) // k => 4 * k + 5

seq: Node = Node(0,Node(1,Node(2,Node(3,Last))))
f: LinFun = LinFun(2, 3)
g: LinFun = LinFun(4, 5)

In [45]:
val res0 = (seq map f) map g        // slow version

res0: IntSeq = Node(17,Node(25,Node(33,Node(41,Last))))

In [46]:
val res1 = seq map (f andThen g)     // fast version

res1: IntSeq = Node(17,Node(25,Node(33,Node(41,Last))))

Мы можем либо 

1. ДВА раза перебрать все элементы списка (ДВА раза пройтись по памяти)
2. и ДВА раза выполнить арифметические операции (* и +)

либо построить композицию f andThen g и

1. ОДИН раз перебирать все элементы списка
2. и ОДИН раз выполнить арифметические операции


# Что такое контравариантный функтор

Напомню, что ковариантным функтором называется всякий класс X, который имеет метод с определенной сигнатурой (условно называемый map) и подчиняющийся определенным правилам (Identity Law, Composition Law).

```
trait X[T] {
  def map[R](f: T => R): X[R]
}
```


В свою очередь контравариантным функтором называется всякий класс X, который имеет метод (условно называемый contramap) с определенной сигнатурой и подчиняющийся определенным правилам (они тоже называются Identity Law, Composition Law)

```
trait X[T] {
  def contramap[R](f: R => T): X[R]
}
```

В этом месте недоуменный читатель может остановиться. Постойте, но если у нас есть контейнер, содержащий T и мы получаем функцию f: T => R, то понятно каким образом мы получаем контейнер с R. Передаем функцию контейнеру, тот погружает функцию внутрь себя и не извлекая элемент применяет к нему функцию. Однако совершенно непонятно, как, имея контейнер с T и получив функцию f: R => T, применить ее в «обратном порядке»?!

В математике в общем виде не у всякой функции есть обратная и нет общего способа найти обратную даже когда она существует. В программировании же нам необходимо действовать конструктивно (не просто работать с существованием, единственность,… но строить и исполнять конструкции) — надо каким-то образом по функции f: R => T построить функцию g: T => R что бы применить ее к содержимому контейнера!

И вот тут оказывается, что наша метафора (ковариантный функтор ~ контейнер) не работает. Давайте разберем почему.

Всякий контейнер предполагает две операции
put — поместить элемент в контейнер
get — извлечь элемент из контейнера

однако у рассмотренных примеров (Option, Try, Future, List, Parser) в той или иной мере есть метод get, но нет метода put! В Option/Try/Future элемент попадает в конструкторе (или в методе apply от companion object) или же в результате какого-то действия. В Parser вообще нельзя попасть, так как Parser[T] — «перерабатывает» строки в T. Parser[T] — это источник T, а не хранилище!

<b>
Ковариантный функтор — это половина контейнера. Та часть, которая отвечает за извлечение данных.</b>

## Примеры контравариантных функторов


Для поиска примеров контравариантных функторов в стандартной библиотеке Scala нам надо забыть про метафору контейнера и искать тип с одним type parameter, который только принимает данные в виде аргументов, но не возвращает в виде результата функции.

In [1]:
import scala.math.Ordering._

import scala.math.Ordering._

In [7]:
import scala.math.Equiv

import scala.math.Equiv

## Ordering

In [2]:
val strX: Ordering[String] = String

strX: Ordering[String] = scala.math.Ordering$String$@296c9cef

In [3]:
val f: (Int => String) = _.toString

f: Int => String = <function1>

In [4]:
val intX: Ordering[Int] = strX on f

intX: Ordering[Int] = scala.math.Ordering$$anon$5@32f7355d

## Equiv

In [8]:
import java.lang.String.CASE_INSENSITIVE_ORDER
import scala.math.Equiv.{fromFunction, fromComparator}

import java.lang.String.CASE_INSENSITIVE_ORDER

import scala.math.Equiv.{fromFunction, fromComparator}

In [9]:
val strX: Equiv[String] = fromComparator(CASE_INSENSITIVE_ORDER)
val f: (Int => String) = _.toString
val intX: Equiv[Int] = fromFunction((x, y) => strX.equiv(f(x), f(y)))

strX: Equiv[String] = scala.math.Equiv$$anon$3@25e0feed
f: Int => String = <function1>
intX: Equiv[Int] = scala.math.Equiv$$anon$4@274f015a

Как и в случае с ковариантным функтором, контравариантному функтору кроме метода с сигнатурой необходимо следовать двум правилам.

Первое правило (Identity Law) гласит: для всякого контравариантного функтора fun должно выполняться IdentityLaw.case0(fun) тождественно равно IdentityLaw.case1(fun)

```object IdentityLaw {
  def case0[T](fun: Contravariant[T]): Contravariant[T] = identity(fun)  
  def case1[T](fun: Contravariant[T]): Contravariant[T] = fun.contramap(identity)
}```


То есть отображение контравариантного функтора единичной функцией не меняет его.

Второе правило (Composition Law) гласит: для всякого контравариантного функтора fun[T] и произвольной пары функций f: Q => R и g: R => T пары должно быть IdentityLaw.case0(fun) тождественно равно IdentityLaw.case1(fun)

```object CompositionLaw {
  def case0[Q, R, T](fun: Contravariant[T], f: Q => R, g: R => T): Contravariant[Q] =
    (fun contramap g) contramap f
  def case1[Q, R, T](fun: Contravariant[T], f: Q => R, g: R => T): Contravariant[Q] =
    fun contramap (f andThen g)
}```

То есть отображение контравариантного функтора последовательно парой функций эквивалентно единичному отображению композицией функций (инвертированной).



Понятия ко- и контра-вариантного функтора являются только стартовой точкой для серьезного изучения применения абстракций из теории категорий в функциональном программировании (в терминах Scala — переход к использованию библиотек Scalaz, Cats). 

Дальнейшие шаги включают:
1. Изучение композиций ко- и контра- вариантных функторов (BiFunctor, ProFunctor, Exponential (Invariant) Functor)
2. Изучение более специализированных конструкций (Applicative Functor, Arrow, Monad), которые уже действительно составляют новую парадигму работы с вычислениями, вводом-выводом, обработкой ошибок, мутирующим состоянием. Укажу хотя бы на то, что всякая монада является ковариантным функтором.

Литература для создания тетрадки

1. [Functors and things using Scala](http://blog.tmorris.net/posts/functors-and-things-using-scala/index.html)
2. [Функторы, аппликативные функторы и монады в картинках](https://habrahabr.ru/post/183150/)
3. [First steps with monads in Scala](https://darrenjw.wordpress.com/2016/04/15/first-steps-with-monads-in-scala/)
4. [FP на Scala: Что такое функтор?](https://habrahabr.ru/company/golovachcourses/blog/266905/)
5. [Scala's Either, Try and the M word](https://mauricio.github.io/2014/02/17/scala-either-try-and-the-m-word.html)

<p style='font-size: 10px'>[Если вы классный](http://homepages.inf.ed.ac.uk/wadler/papers/marktoberdorf/baastad.pdf)</p>

<p style='font-size: 10px'>[Очень-очень классный](https://bartoszmilewski.com/2014/10/28/category-theory-for-programmers-the-preface/)</p>
